In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'distilbert-base-uncased',
    'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, 
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.12xlarge' # ec2 instance type
)

In [ ]:
predictor.endpoint_name

In [ ]:
predictor.predict({
    'inputs': "I am super happy right now."
})

In [ ]:
import boto3
import json
client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {'inputs': "I am super happy right now."}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
response = client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType=content_type,
    Body=payload)
result = response['Body'].read()
result

In [ ]:
#!pip install locust

In [ ]:
!which locust

In [ ]:
%%sh
cd locust
./distributed.sh

In [ ]:
import pandas as pd
locust_data = pd.read_csv('locust/results_stats.csv')
for index, row in locust_data.head(n=2).iterrows():
     print(index, row)